### Magnetometer
If we define $R$ as a rotation matrix from inertial (NED) to body, then we have $$M_B = R^{T}M_G$$ We know that the global magnetic field $M_G$ will be $$M_G=R_\text{offset}
\begin{bmatrix}
1 \\
0 \\ 
0
\end{bmatrix}
$$
Assuming the euler angles are **extrinsic** in $R$, we define the order of rotation as $$R=R_{\psi}R_{\theta}R_{\phi}$$ with $$\text{Roll}=\phi, \text{Pitch}=\theta, \text{Yaw}=\psi$$
Finally $$R^{T} = R_{\phi}^{T}R_{\theta}^{T}R_{\psi}^{T}$$

### Accelerometer
Similarly, $$A_B = R^{T}A_G$$ with the gravity vector pointing upward in the NED frame $$A_G=\begin{bmatrix} 0 \\ 0 \\ -g \end{bmatrix}$$ This gives us $$\begin{bmatrix} a_x \\ a_y \\ a_z \end{bmatrix} = \begin{bmatrix} g\sin{\theta} \\ -g\sin{\phi}\cos{\theta} \\ -g\cos{\phi}\cos{\theta} \end{bmatrix}$$ and therefore $$\theta = \sin^{-1}{(\frac{a_x}{g})}$$ and $$\phi = \tan^{-1}{(\frac{a_y}{a_z})}$$

In [3]:
import casadi as ca
import numpy as np
import math

In [41]:
roll = ca.SX.sym('roll')
pitch = ca.SX.sym('pitch')
yaw = ca.SX.sym('yaw')

r_roll = ca.SX_eye(3)
r_roll[1:,1:] = np.array([[ca.cos(roll), -ca.sin(roll)],
                 [ca.sin(roll), ca.cos(roll)]])
r_pitch = ca.SX_eye(3)
r_pitch[[0,2],[0,2]] = np.array([[ca.cos(pitch), ca.sin(pitch)],
                 [-ca.sin(pitch), ca.cos(pitch)]])
r_yaw = ca.SX_eye(3)
r_yaw[:2,:2] = np.array([[ca.cos(yaw), -ca.sin(yaw)],
                 [ca.sin(yaw), ca.cos(yaw)]])

print("ROLL", r_roll)
print("PITCH", r_pitch)
print("YAW", r_yaw)

ROLL 
[[1, 00, 00], 
 [00, cos(roll), (-sin(roll))], 
 [00, sin(roll), cos(roll)]]
PITCH 
[[cos(pitch), 00, sin(pitch)], 
 [00, 1, 00], 
 [(-sin(pitch)), 00, cos(pitch)]]
YAW 
[[cos(yaw), (-sin(yaw)), 00], 
 [sin(yaw), cos(yaw), 00], 
 [00, 00, 1]]


In [65]:
r_t = (r_roll.T @ r_pitch.T @ r_yaw.T)
print("R.T")
for i in range(0,3):
    for j in range(0,3):
        print(r_t[i, j],end=',')
    print()

R.T
(cos(pitch)*cos(yaw)),(cos(pitch)*sin(yaw)),(-sin(pitch)),
(((sin(roll)*sin(pitch))*cos(yaw))-(cos(roll)*sin(yaw))),(((sin(roll)*sin(pitch))*sin(yaw))+(cos(roll)*cos(yaw))),(sin(roll)*cos(pitch)),
(((cos(roll)*sin(pitch))*cos(yaw))+(sin(roll)*sin(yaw))),(((cos(roll)*sin(pitch))*sin(yaw))-(sin(roll)*cos(yaw))),(cos(roll)*cos(pitch)),


In [58]:
## Compute R_Offset
rot_y = np.array([
        [math.cos(68*math.pi/180), 0, math.sin(68*math.pi/180)],
        [0, 1, 0],
        [-math.sin(68*math.pi/180), 0, math.cos(68*math.pi/180)]], dtype=np.float64)

## Rotation about z-axis
rot_z = np.array([
            [math.cos(11*math.pi/180), -math.sin(11*math.pi/180), 0],
            [math.sin(11*math.pi/180), math.cos(11*math.pi/180), 0],
            [0, 0, 1]], dtype=np.float64)


## We rotate extrinsically here
print(rot_z @ rot_y)

[[ 0.36772402 -0.190809    0.91014888]
 [ 0.07147831  0.98162718  0.17691502]
 [-0.92718385  0.          0.37460659]]
